# 7.4 항공 사진 내 선인장 식별 경진대회 모델 성능 개선

- [항공 사진 내 선인장 식별 경진대회 링크](https://www.kaggle.com/c/aerial-cactus-identification)
- [모델링 노트북 참고 링크](https://www.kaggle.com/bonhart/simple-cnn-on-pytorch-for-beginers)

In [1]:
import pandas as pd

# 데이터 경로
data_path = '/kaggle/input/aerial-cactus-identification/'

labels = pd.read_csv(data_path + 'train.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [2]:
from zipfile import ZipFile

# 훈련 이미지 데이터 압축 풀기
with ZipFile(data_path + 'train.zip') as zipper:
    zipper.extractall()
    
# 테스트 이미지 데이터 압 풀기
with ZipFile(data_path + 'test.zip') as zipper:
    zipper.extractall()

In [3]:
import torch # 파이토치 
import random
import numpy as np

# 시드 값 고정
def set_seed(seed):
#     torch.use_deterministic_algorithms(True)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)

set_seed(10)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
from sklearn.model_selection import train_test_split

# 훈련 데이터, 검증 데이터 분리
train, valid = train_test_split(labels, 
                                test_size=0.1,
                                stratify=labels['has_cactus'],
                                random_state=10)

In [6]:
import cv2
from torch.utils.data import Dataset # 데이터 생성을 위한 클래스

class ImageDataset(Dataset):
    # 초기화 메서드(생성자)
    def __init__(self, df, img_dir = './', transform=None):
        super().__init__() # 상속받은 Dataset의 __init__() 메서드 호출 
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
    
    # 데이터 세트 크기 반환 메서드 
    def __len__(self):
        return len(self.df)
    
    # 인덱스(idx)에 해당하는 데이터 반환 메서드 
    def __getitem__(self, idx):
        img_path = self.img_dir + self.df.iloc[idx, 0] # 이미지 파일 경로 
        image = cv2.imread(img_path) # 이미지 파일 읽기 
        label = self.df.iloc[idx, 1] # 이미지 레이블(타깃 값)
        # 이미지 변환
        if self.transform:
            image = self.transform(image)
        return image, label

In [7]:
import torchvision.transforms as transforms # 이미지 변환을 위한 모듈

transform = transforms.Compose([transforms.ToPILImage(),
                                transforms.Pad(32, padding_mode='symmetric'),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomVerticalFlip(),
                                transforms.RandomRotation(10),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                     std=[0.2, 0.2, 0.2])])

In [8]:
dataset_train = ImageDataset(df=train, img_dir='train/', transform=transform)
dataset_valid = ImageDataset(df=valid, img_dir='train/', transform=transform)

In [9]:
from torch.utils.data import DataLoader # 데이터 로더 생성을 위한 클래스

loader_train = DataLoader(dataset=dataset_train, batch_size=64, 
                          shuffle=True)
loader_valid = DataLoader(dataset=dataset_valid, batch_size=64, 
                          shuffle=False)

In [10]:
import torch.nn as nn # 신경망 모듈
import torch.nn.functional as F # 신경망 모듈에서 자주 사용되는 함수

class Model(nn.Module):
    # 신경망 계층 정의
    def __init__(self):
        super().__init__() # 상속받은 nn.Module의 __init__() 메서드 호출
        # 첫 번째 ~ 다섯 번째 합성곱, 배치 정규화, 최대 풀링 계층 
        self.layer1 = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=32,
                                              kernel_size=3, padding=2),
                                    nn.BatchNorm2d(32), # 배치 정규화 ---①
                                    nn.LeakyReLU(), # LeakyReLU 활성화 함수 ---②
                                    nn.MaxPool2d(kernel_size=2))

        self.layer2 = nn.Sequential(nn.Conv2d(in_channels=32, out_channels=64,
                                              kernel_size=3, padding=2),
                                    nn.BatchNorm2d(64),
                                    nn.LeakyReLU(),
                                    nn.MaxPool2d(kernel_size=2))
        
        self.layer3 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=128,
                                              kernel_size=3, padding=2),
                                    nn.BatchNorm2d(128),
                                    nn.LeakyReLU(),
                                    nn.MaxPool2d(kernel_size=2))
        
        self.layer4 = nn.Sequential(nn.Conv2d(in_channels=128, out_channels=256,
                                              kernel_size=3, padding=2),
                                    nn.BatchNorm2d(256),
                                    nn.LeakyReLU(),
                                    nn.MaxPool2d(kernel_size=2))
        
        self.layer5 = nn.Sequential(nn.Conv2d(in_channels=256, out_channels=512,
                                              kernel_size=3, padding=2),
                                    nn.BatchNorm2d(512),
                                    nn.LeakyReLU(),
                                    nn.MaxPool2d(kernel_size=2))
        # 평균 풀링 계층 
        self.avg_pool = nn.AvgPool2d(kernel_size=4) 
        # 전결합 계층 ---③
        self.fc1 = nn.Linear(in_features=512 * 1 * 1, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=2)

    # 순전파 출력 정의 
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.avg_pool(x)
        x = x.view(-1, 512 * 1 * 1) # 평탄화
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [11]:
model = Model().to(device)

model

Model(
  (layer1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer4): Sequential(
    (0): Conv2d(128,

In [12]:
# 손실함수
criterion = nn.CrossEntropyLoss()

In [13]:
# 옵티마이저
optimizer = torch.optim.Adamax(model.parameters(), lr=0.0002)

In [14]:
epochs = 35 # 총 에폭

# 총 35에폭만큼 훈련
for epoch in range(epochs):
    # 미니배치 크기만큼 데이터를 추출하는 작업을 '반복 횟수'만큼 반복 
    for i, (images, labels) in enumerate(loader_train):
        # 이미지, 레이블 데이터 미니배치를 장비에 할당 
        images = images.to(device)
        labels = labels.to(device)
        
        # 옵티마이저 내 기울기 초기화
        optimizer.zero_grad()
        # 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 기존에 정의한 손실함수를 활용해 outputs와 labels의 손실값 계산
        loss = criterion(outputs, labels)
        # 오차 역전파 수행
        loss.backward()
        # 가중치 갱신
        optimizer.step()
        
    print (f'에폭: [{epoch+1}/{epochs}], 손실값: {loss.item():.4f}')

에폭: [1/35], 손실값: 0.1571
에폭: [2/35], 손실값: 0.0014
에폭: [3/35], 손실값: 0.0248
에폭: [4/35], 손실값: 0.1070
에폭: [5/35], 손실값: 0.0004
에폭: [6/35], 손실값: 0.0008
에폭: [7/35], 손실값: 0.0001
에폭: [8/35], 손실값: 0.0002
에폭: [9/35], 손실값: 0.0007
에폭: [10/35], 손실값: 0.0060
에폭: [11/35], 손실값: 0.1186
에폭: [12/35], 손실값: 0.0082
에폭: [13/35], 손실값: 0.0006
에폭: [14/35], 손실값: 0.0001
에폭: [15/35], 손실값: 0.0328
에폭: [16/35], 손실값: 0.0001
에폭: [17/35], 손실값: 0.0440
에폭: [18/35], 손실값: 0.0000
에폭: [19/35], 손실값: 0.0824
에폭: [20/35], 손실값: 0.0707
에폭: [21/35], 손실값: 0.0178
에폭: [22/35], 손실값: 0.0001
에폭: [23/35], 손실값: 0.0000
에폭: [24/35], 손실값: 0.2162
에폭: [25/35], 손실값: 0.0006
에폭: [26/35], 손실값: 0.0023
에폭: [27/35], 손실값: 0.0017
에폭: [28/35], 손실값: 0.0000
에폭: [29/35], 손실값: 0.0000
에폭: [30/35], 손실값: 0.0000
에폭: [31/35], 손실값: 0.0000
에폭: [32/35], 손실값: 0.4657
에폭: [33/35], 손실값: 0.0000
에폭: [34/35], 손실값: 0.0223
에폭: [35/35], 손실값: 0.0000


In [15]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수

roc_auc = 0 # ROC AUC 초기화

model.eval() # 모델을 평가 상태로 설정 

with torch.no_grad(): # 기울기 계산 비활성
    for images, labels in loader_valid:
        # 이미지, 레이블 데이터 미니배치를 장비에 할당 
        images = images.to(device)
        labels = labels.to(device)
        # 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        preds = torch.softmax(outputs.cpu(), dim=1)[:, 1] # 예측 확률값
        true = labels.cpu() # 실제값
        # 검증 데이터 ROC AUC 점수 계산 
        roc_auc += roc_auc_score(true, preds) / len(loader_valid)

    print(f'검증 데이터 ROC AUC : {roc_auc}')

검증 데이터 ROC AUC : 0.9999999999999997


In [16]:
dataset_test = ImageDataset(df=submission, img_dir='test/', transform=transform)
loader_test = DataLoader(dataset=dataset_test, batch_size=64, 
                         shuffle=False)

In [17]:
model.eval() # 모델을 평가 상태로 설정

preds = [] # 타깃 예측 값 저장용 변수 초기화

with torch.no_grad(): # 기울기 계산 비활성
    for images, _ in loader_test:
        # 이미지 데이터 미니배치를 장비에 할당
        images = images.to(device)
        # 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 타깃 값이 1일 확률(예측 값)
        preds_temp = torch.softmax(outputs.cpu(), dim=1)[:, 1].tolist()
        # preds에 preds_temp 이어붙이기
        preds.extend(preds_temp)

In [18]:
submission['has_cactus'] = preds
submission.to_csv('submission.csv', index=False)

In [19]:
submission

,id,has_cactus
0,000940378805c44108d287872b2f04ce.jpg,0.999992
1,0017242f54ececa4512b4d7937d1e21e.jpg,1.000000
2,001ee6d8564003107853118ab87df407.jpg,0.000003
3,002e175c3c1e060769475f52182583d0.jpg,0.000003
4,0036e44a7e8f7218e9bc7bf8137e4943.jpg,0.037364
...,...,...
3995,ffaafd0c9f2f0e73172848463bc2e523.jpg,0.999968
3996,ffae37344310a1549162493237d25d3f.jpg,1.000000
3997,ffbd469c56873d064326204aac546e0d.jpg,1.000000
3998,ffcb76b7d47f29ece11c751e5f763f52.jpg,1.000000


In [20]:
import shutil

shutil.rmtree('./train')
shutil.rmtree('./test')